<h1> <center> Será que um professor fica por 2 anos em uma escola? </center> </h1>



<img src='https://media.giphy.com/media/xT5LMpjrnf2Ao4hBMk/giphy.gif'/>




In [14]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pandas as pd
import re
import numpy as np
from matplotlib import pyplot as plt
import os
from glob import glob
from pathlib import Path
import seaborn as sns
from unicodedata import normalize
from tqdm import tqdm
from scipy import stats
from sklearn import metrics
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#Arquivos criados para deixar o codigo do notebook mais clean.
from files_util import *
from df_inep_utils import *
from SetorCensitario import *
from modelagem import *

In [4]:
#alguns numeros ficam muito grandes.
pd.options.display.float_format = '{:.2f}'.format

## <center> Vamos começar a jornada</center>
<img src='https://media.giphy.com/media/fdHg7T902uzLy/giphy.gif'/>

## 01. Obtencao dos dados brutos

In [5]:
#download dos dados do censo
#list(baixa_inep('micro_censo_escolar',anos_pesquisa))
#Rodei uma vez e comentei para nao sobrescrever 5.3Gb por engano

In [6]:
#extraindo arquivos baixados
#funcoes de file_utils
shutil.rmtree(ARQ_PATH,ignore_errors=True)
os.mkdir(ARQ_PATH)
list(map(extrai_arq,glob(f'{CSV_PATH}*.zip')));

patool: Extracting /home/itsbarreto/csv/educacao/micro_censo_escolar_2013.zip ...
patool: ... /home/itsbarreto/csv/educacao/micro_censo_escolar_2013.zip extracted to `/home/itsbarreto/csv/educacao/TMP/'.
patool: Extracting /home/itsbarreto/csv/educacao/TMP/2013/DADOS/TURMAS.rar ...
patool: running /usr/bin/unrar x -- /home/itsbarreto/csv/educacao/TMP/2013/DADOS/TURMAS.rar
patool:     with cwd='/home/itsbarreto/csv/educacao/dsc/2013'
patool: ... /home/itsbarreto/csv/educacao/TMP/2013/DADOS/TURMAS.rar extracted to `/home/itsbarreto/csv/educacao/dsc/2013'.
patool: Extracting /home/itsbarreto/csv/educacao/TMP/2013/DADOS/MATRICULA_CO.rar ...
patool: running /usr/bin/unrar x -- /home/itsbarreto/csv/educacao/TMP/2013/DADOS/MATRICULA_CO.rar
patool:     with cwd='/home/itsbarreto/csv/educacao/dsc/2013'
patool: ... /home/itsbarreto/csv/educacao/TMP/2013/DADOS/MATRICULA_CO.rar extracted to `/home/itsbarreto/csv/educacao/dsc/2013'.
patool: Extracting /home/itsbarreto/csv/educacao/TMP/2013/DADOS/ES

## 02. Vamos ver os dados demográficos dos docentes da rede pública?

### Vamos comecar com o Centro Oeste que  é o menor arquivo

In [12]:
[print(c) for c in df17.columns];

NU_ANO_CENSO
CO_PESSOA_FISICA
NU_DIA
NU_MES
NU_ANO
NU_IDADE_REFERENCIA
NU_IDADE
TP_SEXO
TP_COR_RACA
TP_NACIONALIDADE
CO_PAIS_ORIGEM
CO_UF_NASC
CO_MUNICIPIO_NASC
CO_UF_END
CO_MUNICIPIO_END
TP_ZONA_RESIDENCIAL
IN_NECESSIDADE_ESPECIAL
IN_CEGUEIRA
IN_BAIXA_VISAO
IN_SURDEZ
IN_DEF_AUDITIVA
IN_SURDOCEGUEIRA
IN_DEF_FISICA
IN_DEF_INTELECTUAL
IN_DEF_MULTIPLA
TP_ESCOLARIDADE
TP_NORMAL_MAGISTERIO
TP_SITUACAO_CURSO_1
CO_AREA_CURSO_1
CO_CURSO_1
IN_LICENCIATURA_1
IN_COM_PEDAGOGICA_1
NU_ANO_INICIO_1
NU_ANO_CONCLUSAO_1
TP_TIPO_IES_1
CO_IES_1
TP_SITUACAO_CURSO_2
CO_AREA_CURSO_2
CO_CURSO_2
IN_LICENCIATURA_2
IN_COM_PEDAGOGICA_2
NU_ANO_INICIO_2
NU_ANO_CONCLUSAO_2
TP_TIPO_IES_2
CO_IES_2
TP_SITUACAO_CURSO_3
CO_AREA_CURSO_3
CO_CURSO_3
IN_LICENCIATURA_3
IN_COM_PEDAGOGICA_3
NU_ANO_INICIO_3
NU_ANO_CONCLUSAO_3
TP_TIPO_IES_3
CO_IES_3
IN_DISC_QUIMICA
IN_DISC_FISICA
IN_DISC_MATEMATICA
IN_DISC_BIOLOGIA
IN_DISC_CIENCIAS
IN_DISC_LINGUA_PORTUGUESA
IN_DISC_LINGUA_INGLES
IN_DISC_LINGUA_ESPANHOL
IN_DISC_LINGUA_FRANCES
IN_D

In [17]:

def analisa_inicio(dfa):
    re_id_hist = re.compile('(^CO_[A-Z]).+(\D$)')
    cols_hist = [c for c in dfa.columns if re_id_hist.match(c) or c.startswith('NU') or c.startswith('IN') or c in ['TP_NACIONALIDADE','TP_COR_RACA']]
    for c  in cols_hist:
        display(c)
        s = dfa[c]
        if s.nunique() > 10:
            display(s.describe())
        else:
            display(s.value_counts())
        s.hist()
        plt.show();
    display(dfa['TP_SEXO'].value_counts())
    dfa['TP_SEXO'].value_counts().plot.bar();
    plt.show()

df17 = monta_df_docente(f'{ARQ_PATH}/2017/DOCENTES_CO.CSV')
#dfa17 = monta_df_professores(df17.loc[df17.TP_TIPO_DOCENTE == 1])
print(f'O Centro Oeste possuia em 2017 {df17.loc[df17.TP_TIPO_DOCENTE == 1].CO_PESSOA_FISICA.nunique()} professores.')
#analisa_inicio(dfa17)


NU_ANO_CENSO: int16
CO_PESSOA_FISICA: int64
NU_DIA: int8
NU_MES: int8
NU_ANO: int16
NU_IDADE_REFERENCIA: int8
NU_IDADE: int8
TP_SEXO: int8
TP_COR_RACA: int8
TP_NACIONALIDADE: int8
CO_PAIS_ORIGEM: int16
CO_UF_NASC: float16
CO_MUNICIPIO_NASC: float32
CO_UF_END: float16
CO_MUNICIPIO_END: float32
TP_ZONA_RESIDENCIAL: float16
O Centro Oeste possuia em 2017 167113 professores.


### Vamos ver se houve diferença entre 2011 e 2017
#### Para isso temos que fazer um de/para nos campos

Utilizei 2017 como base pois tem a informacao se o campos esta disponivel em outros anos.

In [ ]:

ano_ref = '2017'
campos_docentes = {}
for a in anos_pesquisa:
    campos_docentes[a] = importa_campos(f'{ARQ_PATH}{a}/dic_dados_docentes.csv')


Tipos de campos:

* Não havia em algum dos anos esolhidos: serao deletados.
* Com o mesmo nome: OK
* Com nome diferente: descricao igual
* Com nome e descricao diferente: nome proximo (preferencialmente a partir do primeiro _ ).


In [ ]:
def campos_ref(dref,anos):
    duso = dref.loc[dref[f'IN_{anos_pesquisa[0][2:]}']=='s']
    for a in anos[1:]:
        duso = duso.loc[duso[f'IN_{a[2:]}'] == 's']
    return duso

campos_docentes[ano_ref]['NM_ANT_VAR'] = campos_docentes[ano_ref].merge(campos_docentes['2015'],right_index=True,left_index=True,how='left')['NOME_ANTIGO']
display(campos_docentes[ano_ref].head())
duso = campos_ref(campos_docentes[ano_ref],anos_pesquisa)[['DESCRICAO','NM_ANT_VAR']]
display(duso.shape)
display(duso.head())

In [ ]:
t = duso.copy()
for k,v in campos_docentes.items():
    if k != ano_ref:
        t[f'NM_VAR_{k}'] = monta_de_para(duso,v[['DESCRICAO']],k)
d = t.copy()

In [ ]:
display(d.shape)
display(d.head())


In [ ]:
variaveis_docentes = d.dropna()[[f'NM_VAR_{a}' for a in anos_pesquisa if a!=ano_ref]].reset_index()
variaveis_docentes.columns = [c if c!='VARIAVEL' else f'NM_VAR_{ano_ref}' for c in variaveis_docentes.columns]

In [ ]:
display(variaveis_docentes.shape)
display(variaveis_docentes)


### Professores 

In [ ]:
prof_df = pd.DataFrame(columns=list(variaveis_docentes[f'NM_VAR_{ano_ref}'].values) + ['ANO'])

for k,v in tqdm(campos_docentes.items()):
    cpos = v
    d = carrega_arquivo_inep(f'{ARQ_PATH}{k}/DOCENTES_CO.CSV',cols=variaveis_docentes[f'NM_VAR_{k}'].values)
    col_uf = 'CO_UF' if 'CO_UF' in d.columns else 'FK_COD_ESTADO'
    d = d.loc[d[col_uf] == 53]
    d.columns = variaveis_docentes[f'NM_VAR_{ano_ref}'].values
    d['ANO'] = int(k)
    prof_df = prof_df.append(d)
    
    

In [ ]:
prof_df.fillna(-1,inplace=True)
prof_df = ajusta_colunas_int_df_inep(prof_df)
prof_df = prof_df[prof_df.TP_TIPO_DOCENTE == 1]
prof_df.head()

In [ ]:
list(map(print,enumerate(prof_df.columns)));

In [ ]:
pss_ano.set_index(['CO_PESSOA_FISICA','ANO'],inplace=True)

In [ ]:
#pss_ano['NUM_QTD_ESCOLAS'] = prof_df.groupby(['CO_PESSOA_FISICA','ANO'])['CO_ENTIDADE'].nunique().fillna(0)
#pss_ano['IN_LECIONA_PUBLICA'] = (prof_df.groupby(['CO_PESSOA_FISICA','ANO'])['TP_DEPENDENCIA'].max() <= 4).fillna(False).astype(np.int8)
pss_ano['NUM_QTD_TURMAS'] = prof_df.groupby(['CO_PESSOA_FISICA','ANO'])['ID_TURMA'].count().fillna(0)


In [ ]:
pss_ano = monta_df_prof_ano(prof_df)

In [ ]:
a = pss_ano.fillna(-1)
pss_ano = ajusta_colunas_int_df_inep(a)

In [ ]:
prof_df.to_csv(f'{PATH_DF_CSV}prof_df.csv',index=False)
pss_ano.to_csv(f'{PATH_DF_CSV}pessoas_prof_df.csv',index=False)

<table columns=2 style='width:100%'><tr><td>
<img src='img/cp_icon.png'/></td><td> <h2>Check-point, recarregando os DataFrames para analisar a situacao do DF e montar a target.</h2></td></tr></table>

In [ ]:
prof_df = pd.read_csv(f'{PATH_DF_CSV}prof_df.csv',low_memory=False)
pss_ano= pd.read_csv(f'{PATH_DF_CSV}pessoas_prof_df.csv',low_memory=False)

In [ ]:
display(pss_ano.reset_index().ANO.value_counts())
pss_ano.shape

In [ ]:
pss_ano.reset_index(inplace=True)
pss_ano = ajusta_colunas_int_df_inep(pss_ano,vai_printar_cols=False)

#### Curiosidade para ver o comportamento das variaveis

In [ ]:
for c in pss_ano.columns[2:]:
    if not c.startswith('CO'):
        for a in anos_pesquisa:
            display(a + ' - ' + c)
            if c.startswith('NU'):
                display(pss_ano.loc[pss_ano.ANO == int(a)][c].describe())
                pss_ano.loc[pss_ano.ANO == int(a)][c].hist()
            else:
                display(pss_ano.loc[pss_ano.ANO == int(a)][c].value_counts())
                pss_ano.loc[pss_ano.ANO == int(a)][c].value_counts().plot.bar()
            plt.show()
            #sns.barplot()
    


### Taxa de rotatividade de professores entre 2 anos .  




In [ ]:
def monta_saidas(pt, ano,turmas=[]):
    pe_ano = pt.loc[(pt.TP_DEPENDENCIA < 4) & (pt.ANO == int(ano)) & ((len(turmas)==0) | (pt.CO_ENTIDADE.isin(turmas)))][['CO_PESSOA_FISICA','CO_ENTIDADE']].drop_duplicates()
    pe_fut = pt.loc[(pt.TP_DEPENDENCIA < 4) & ((len(turmas)==0) | (pt.CO_ENTIDADE.isin(turmas))) & (pt.ANO == int(anos_pesquisa[anos_pesquisa.index(str(ano))+1])) & pt.CO_PESSOA_FISICA.isin(pe_ano.CO_PESSOA_FISICA.unique())][['CO_PESSOA_FISICA','CO_ENTIDADE']].drop_duplicates()
    pe_ano = pe_ano.loc[pe_ano.CO_PESSOA_FISICA.isin(pe_fut.CO_PESSOA_FISICA.unique())]
    to = pe_ano.merge(pe_fut,indicator=True,on=['CO_PESSOA_FISICA','CO_ENTIDADE'],how='outer')
    #vamos manter somente as escolas que estavam abertas nos 2 periodos
    to = to.loc[to._merge =='left_only']
    return to[['CO_PESSOA_FISICA','CO_ENTIDADE']],pe_ano,pe_fut


In [ ]:

for ano in anos_pesquisa[:-1]:
    t,a,f = monta_saidas(prof_df,ano)
    p = t.shape[0]/a.shape[0] * 100
    display(f'{ano}/{int(ano)+2} havia {a.shape[0]} professores/escola, houve {t.shape[0]} saidas ({p}%)');


### Como  o turnover total foi estável no período iremos modelar somente com o biênio 2015/2017

##### Importando dados de escolas e turmas de escolas publicas em funcionamento do DF

In [ ]:
def imp_esc_dist_atv_df(ano):
    d_esc = pd.read_csv(f'{ARQ_PATH}{ano}/ESCOLAS.CSV',sep='|',encoding='latin1',low_memory=False)
    d_esc.fillna(-1,inplace=True)
    d_esc = ajusta_colunas_int_df_inep(d_esc,vai_printar_cols=False)
    d_esc['ANO'] = ano
    return d_esc.loc[(d_esc.CO_UF == 53) & (d_esc.TP_SITUACAO_FUNCIONAMENTO == 1) & (d_esc.TP_DEPENDENCIA == 2)]

escolas = imp_esc_dist_atv_df(2017).append(imp_esc_dist_atv_df(2015))

escolas.head()

In [ ]:
def imp_turma_esc_atv_df(ano,escolas):
    d_tur = pd.read_csv(f'{ARQ_PATH}{ano}/TURMAS.CSV',sep='|',encoding='latin1',low_memory=False)
    d_tur.fillna(-1,inplace=True)
    d_tur = ajusta_colunas_int_df_inep(d_tur,vai_printar_cols=False)
    d_tur['ANO'] = ano
    return d_tur.loc[(d_tur.CO_ENTIDADE.isin(escolas))]


In [ ]:
turmas = imp_turma_esc_atv_df(2015,escolas.loc[escolas.ANO == 2015].CO_ENTIDADE.unique())\
        .append(imp_turma_esc_atv_df(2017,escolas.loc[escolas.ANO == 2017].CO_ENTIDADE.unique()))

In [ ]:
display(turmas.dtypes)
turmas.head()

In [ ]:
escolas.to_csv(f'{PATH_DF_CSV}escolas_distritais_15_17_func_df.csv',index=False)
turmas.to_csv(f'{PATH_DF_CSV}turmas_df.csv',index=False)

In [ ]:
escolas = pd.read_csv(f'{PATH_DF_CSV}escolas_distritais_15_17_func_df.csv',low_memory=False)
turmas = pd.read_csv(f'{PATH_DF_CSV}turmas_df.csv',low_memory=False)

In [ ]:
#escolas distritais que estavam funcionamento em 2015 e 2017
e17 = escolas.loc[(escolas.ANO == 2017)]['CO_ENTIDADE'].unique()
co_e15_17 = escolas.loc[(escolas.ANO == 2015) & (escolas.CO_ENTIDADE.isin(e17))]['CO_ENTIDADE'].unique()
display(len(e17))
display(len(co_e15_17))

In [ ]:
#turnover 2015 em escolas em funcionamento
ano = 2015
t,a,f = monta_saidas(prof_df,ano,turmas=co_e15_17)
p = t.shape[0]/a.shape[0] * 100
display(f'{ano}/{int(ano)+2} havia {a.shape[0]} professores/escola, houve {t.shape[0]} saidas ({p}%)');


### <center>  Temos publico e temos a target! </center>

<img src='https://media.giphy.com/media/3o6MbkNYWUzAuKmDa8/giphy.gif'>




In [ ]:
a = a.merge(t,on=['CO_ENTIDADE','CO_PESSOA_FISICA'],indicator=True,how='left')
a['target'] = a._merge == 'both'
a.drop('_merge',axis=1,inplace=True)
a['ano'] = 2015
display(a.target.value_counts())


In [ ]:
a.to_csv(f'{PATH_DF_CSV}prof_escola_target.csv',index=False)

In [ ]:
#atalho para montagem do professores
def atalho_prof_df2015():
    a = ajusta_colunas_int_df_inep(carrega_arquivo_docentes(f'{ARQ_PATH}2015/DOCENTES_CO.CSV'))
    a['ANO'] = 2015
    b = ajusta_colunas_int_df_inep(carrega_arquivo_docentes(f'{ARQ_PATH}2017/DOCENTES_CO.CSV'))
    b['ANO'] = 2017
    c = a.append(b)
    c.to_csv(f'{PATH_DF_CSV}prof_df.csv',index=None)
    del a, b
    return c
#prof_df = atalho_prof_df2015()


### Apaga arquivos nao utilizados

In [ ]:
import os
for path, subdirs, files in os.walk(ARQ_PATH):
    for name in files:
        if '_' in name and not name.endswith('_CO.CSV') and name.endswith('CSV'):
            f = os.path.join(path, name)
            !rm -rf {f}    

<table columns=2 style='width:100%'><tr><td><img src='img/cp_icon.png'/>
</td><td> <h2>Check-point, importando os DataFrames para criação de features.</h2></td></tr></table>

In [ ]:
escolas = ajusta_colunas_int_df_inep(pd.read_csv(f'{PATH_DF_CSV}escolas_distritais_15_17_func_df.csv',low_memory=False),vai_printar_cols=False)
turmas = ajusta_colunas_int_df_inep(pd.read_csv(f'{PATH_DF_CSV}turmas_df.csv',low_memory=False),vai_printar_cols=False)
prof_df = ajusta_colunas_int_df_inep(pd.read_csv(f'{PATH_DF_CSV}prof_df.csv',low_memory=False),vai_printar_cols=False)
pss_ano = None
try:
    pss_ano= ajusta_colunas_int_df_inep(pd.read_csv(f'{PATH_DF_CSV}pessoas_prof_df.csv',low_memory=False),vai_printar_cols=False)
except Exception as e:
    print(f'Erro na importacao de pss_ano{e}')
    pass
pet = ajusta_colunas_int_df_inep(pd.read_csv(f'{PATH_DF_CSV}prof_escola_target.csv',low_memory=False),vai_printar_cols=False)

## 02. Criação de features para o estudo

- Localizacao:
    * criminalidade
    * linhas de onibus
    * contexto social (setor censitario)
- Infra da escola
    * situacao da escola
- Turma
    * alunos
    * disciplina da turma x displina de formacao




#### Iremos começar com os dados que já temos (escolas e turmas).

- Selecionando features das escolas, professores e turmas.

In [ ]:
list(map(print,prof_df.columns));


In [ ]:
list(map(print,turmas.columns))
cols_sel = [
    'CO_TIPO_ATIVIDADE_1',    'CO_TIPO_ATIVIDADE_2',    'CO_TIPO_ATIVIDADE_3',
'CO_TIPO_ATIVIDADE_4','CO_TIPO_ATIVIDADE_5','CO_TIPO_ATIVIDADE_6',
'ID_TURMA','IN_AUTONOMIA_ESCOLAR','IN_BRAILLE',
'IN_COMUNICACAO_ALT_AUMENT','IN_CONVENIADA_PP',
'IN_DIA_SEMANA_DOMINGO','IN_DIA_SEMANA_QUARTA',
'IN_DIA_SEMANA_QUINTA','IN_DIA_SEMANA_SABADO',
'IN_DIA_SEMANA_SEGUNDA','IN_DIA_SEMANA_SEXTA',
'IN_DIA_SEMANA_TERCA','IN_DISC_ARTES',
'IN_DISC_ATENDIMENTO_ESPECIAIS','IN_DISC_BIOLOGIA',
'IN_DISC_CIENCIAS','IN_DISC_DIVER_SOCIO_CULTURAL',
'IN_DISC_EDUCACAO_FISICA','IN_DISC_ENSINO_RELIGIOSO',
'IN_DISC_ESTUDOS_SOCIAIS','IN_DISC_EST_SOCIAIS_SOCIOLOGIA',
'IN_DISC_FILOSOFIA','IN_DISC_FISICA',
'IN_DISC_GEOGRAFIA','IN_DISC_HISTORIA',
'IN_DISC_INFORMATICA_COMPUTACAO','IN_DISC_LIBRAS',
'IN_DISC_LINGUA_ESPANHOL','IN_DISC_LINGUA_FRANCES',
'IN_DISC_LINGUA_INDIGENA','IN_DISC_LINGUA_INGLES',
'IN_DISC_LINGUA_OUTRA','IN_DISC_LINGUA_PORTUGUESA',
'IN_DISC_MATEMATICA','IN_DISC_OUTRAS',
'IN_DISC_PEDAGOGICAS','IN_DISC_PROFISSIONALIZANTE',
'IN_DISC_QUIMICA','IN_DISC_SOCIOLOGIA',
'IN_EDUCACAO_INDIGENA','IN_EJA',
'IN_ENRIQ_CURRICULAR','IN_ESPECIAL_EXCLUSIVA',
'IN_INFORMATICA_ACESSIVEL','IN_MAIS_EDUCACAO',
'IN_ORIENTACAO_MOBILIDADE',
'IN_PORT_ESCRITA','IN_PROCESSOS_MENTAIS',
'IN_PROFISSIONALIZANTE','IN_RECURSOS_BAIXA_VISAO',
'IN_REGULAR','IN_SINAIS','IN_SOROBAN','NU_DIAS_ATIVIDADE',
'NU_DURACAO_TURMA','NU_MATRICULAS','TX_HR_INICIAL'
]
ft_turmas = turmas.loc[turmas.ANO == 2015][cols_sel].set_index('ID_TURMA')

qtd_turmas = ft_turmas.shape[0]
dc = []
ft_turmas['IN_MANHA'] = (ft_turmas.TX_HR_INICIAL < 12).astype(np.int8)
ft_turmas['IN_TARDE'] = ((ft_turmas.TX_HR_INICIAL >= 12) & (ft_turmas.TX_HR_INICIAL < 18)).astype(np.int8)
ft_turmas['IN_NOITE'] = (ft_turmas.TX_HR_INICIAL >= 18).astype(np.int8)
dc.append('TX_HR_INICIAL')
display(ft_turmas.shape)
for c in ft_turmas.columns:
    if stats.mode(ft_turmas[c])[1] / qtd_turmas > 0.9:
        dc.append(c)
ft_turmas.drop(dc,axis=1,inplace=True)
display(ft_turmas.head())
display(ft_turmas.shape)

In [ ]:
profs = prof_df.loc[prof_df.ANO == 2015][['CO_PESSOA_FISICA','ID_TURMA','CO_ENTIDADE']]


In [ ]:
display(profs.shape)
a = profs.merge(ft_turmas,on='ID_TURMA')
b = a.groupby(['CO_PESSOA_FISICA','CO_ENTIDADE'])[[c for c in a.columns if c.startswith('IN_') or c.startswith('NU_')]].sum()
b.columns = ['SUM_TURMA_' + c for c in b.columns]

d = a.groupby(['CO_PESSOA_FISICA','CO_ENTIDADE'])[[c for c in a.columns if c.startswith('NU_')]].max()
d.columns = ['MAX_TURMA_' + c for c in d.columns]
e = a.groupby(['CO_PESSOA_FISICA','CO_ENTIDADE'])[[c for c in a.columns if c.startswith('NU_')]].min()
e.columns = ['MIN_TURMA_' + c for c in e.columns]

b = b.merge(d,right_index=True,left_index=True)
prof_escolas = b.merge(e,right_index=True,left_index=True)
del a, b, d, e
display(prof_escolas.shape)
prof_escolas.head()

In [ ]:
#so interessa a situacao da escola no momento de analise da saida
list(map(print,escolas.columns));


In [ ]:
ft_escolas = escolas.loc[escolas.NU_ANO_CENSO == 2015]
ft_escolas['NUM_DD_AULAS'] = (pd.to_datetime(ft_escolas.DT_ANO_LETIVO_TERMINO) - pd.to_datetime(ft_escolas.DT_ANO_LETIVO_INICIO)).dt.days
ft_escolas['NUM_DD_AULAS'].head()
cols_sel =['CO_ENTIDADE','CO_ORGAO_REGIONAL','TP_LOCALIZACAO',
           'TP_OCUPACAO_PREDIO_ESCOLAR','TP_OCUPACAO_GALPAO',
          'TP_ATIVIDADE_COMPLEMENTAR']
ft_escolas = ft_escolas[cols_sel +[c for c in ft_escolas.columns if c.startswith('IN_') or c.startswith('NU')]]
ft_escolas.set_index('CO_ENTIDADE',inplace=True)
#vamos as os indicadores que estao em mais de 90% das escolas.
print(ft_escolas.shape)
cols = ft_escolas.columns
qtd_esc = ft_escolas.shape[0]
dc = []
for c in cols:
    if stats.mode(ft_escolas[c])[1] / qtd_esc > 0.9:
        dc.append(c)
ft_escolas.drop(dc,axis=1,inplace=True)
display(ft_escolas.head())
print(ft_escolas.shape)


In [ ]:
display(ft_escolas.dtypes)

In [ ]:
ft_escolas['PC_SALAS_UTILIZADAS'] = (ft_escolas.NU_SALAS_UTILIZADAS/ft_escolas.NU_SALAS_EXISTENTES)\
                                    .astype(np.float16)
ft_escolas['NU_TTL_MATRICLAS'] = turmas.groupby('CO_ENTIDADE').NU_MATRICULAS.sum()
ft_escolas['NU_PROFESSORES'] = prof_df.groupby('CO_ENTIDADE')['CO_PESSOA_FISICA'].count()
cd = []
for c in ft_escolas.columns:
    if c.startswith('NU_') and c!= 'NU_TTL_MATRICLAS':
        ft_escolas[f'{c}_PER_MATRICULA'] = ft_escolas[c]/ft_escolas['NU_TTL_MATRICLAS']
        cd.append(c)
ft_escolas.drop(c,axis=1,inplace=True)
display(ft_escolas.shape)
display(ft_escolas.head())

In [ ]:
prof_escolas = prof_escolas.reset_index().merge(ft_escolas.reset_index(),on='CO_ENTIDADE').set_index(['CO_PESSOA_FISICA','CO_ENTIDADE'])

In [ ]:
display(prof_escolas.shape)
display(prof_escolas.head())

In [ ]:
model_vars = prof_escolas.reset_index().merge(pet.drop('ano',axis=1),on=['CO_PESSOA_FISICA','CO_ENTIDADE'])

In [ ]:
qtd_esc = model_vars.shape[0]
dc = []
for c in model_vars.columns:
    if stats.mode(model_vars[c])[1] / qtd_esc > 0.9:
        dc.append(c)
    else:
        display(c)
        model_vars[c].hist()
        plt.show()
print(dc)

### Dados de:
01. Alunos que compoe as turmas que o professor da aula.


#### Alunos que compoe as turmas

In [ ]:
a = ajusta_colunas_int_df_inep(carrega_arquivo_inep(f'{ARQ_PATH}2015/MATRICULA_CO.CSV'))

In [ ]:
display(a.shape)
display(a.head(2).T)

In [ ]:
alunos = a.loc[a.CO_ENTIDADE.isin(pet.CO_ENTIDADE.unique()) & a.ID_TURMA.isin(profs.ID_TURMA.unique())]
#del a

In [ ]:
alunos['IN_H0MEM'] = (alunos.TP_SEXO == 1).astype(np.int8)
alunos['IN_TERRA_NATAL'] = alunos.CO_MUNICIPIO_NASC == alunos.CO_MUNICIPIO_END
alunos['IN_ONIBUS'] = alunos.TP_RESPONSAVEL_TRANSPORTE.notna()
alunos = alunos.merge(pd.get_dummies(alunos.TP_COR_RACA,prefix='IN_TP_COR_RACA',dtype=np.int8),right_index=True, left_index=True)
display(alunos.shape)
list(map(print,alunos.columns));

In [ ]:
cols_count = ['CO_PESSOA_FISICA']
cols_soma = [f'IN_TP_COR_RACA_{i}' for i in range(0,6)]
cols_media = ['IN_ONIBUS','IN_TERRA_NATAL','IN_H0MEM','NU_IDADE']

cols_gb = ['CO_ENTIDADE','ID_TURMA']
a = alunos.groupby(cols_gb)
c = a[cols_count].count()
c.columns = [f'ALUNO_COUNT_{col}' for col in c.columns]
s = a[cols_soma].sum()
s.columns = [f'ALUNO_SUM_{col}' for col in s.columns]
m = a[cols_media].mean()
m.columns = [f'ALUNO_MEAN_{col}' for col in m.columns]
turma_alunos = c.merge(s,left_index=True,right_index=True).merge(m,left_index=True,right_index=True)
del a,c,s,m
display(turma_alunos.shape)
display(turma_alunos.head())

In [ ]:
turma_alunos.ALUNO_COUNT_CO_PESSOA_FISICA = turma_alunos.ALUNO_COUNT_CO_PESSOA_FISICA.clip_upper(50)
for c in turma_alunos.columns:
    if c.startswith('ALUNO_SUM_IN'):
        turma_alunos[c] = turma_alunos.clip_upper(50)

#### Dados para cruzar professores com turmas

In [ ]:
p15 = ajusta_colunas_int_df_inep(carrega_arquivo_inep(f'{ARQ_PATH}2015/DOCENTES_CO.CSV'))
p15 = p15.loc[p15.CO_PESSOA_FISICA.isin(pet.CO_PESSOA_FISICA)]


In [ ]:
p15r = monta_df_professores_vs17_resu(p15)


In [ ]:
p15r.reset_index(inplace=True)
p15r.columns = [f'PROF_{c}' for c in p15r.columns ]

In [ ]:
list(map(print,p15r.columns));

In [ ]:
a = turma_alunos.reset_index().merge(profs,on=cols_gb)
a.columns = [c for c in a.columns[:-1]] + ['PROF_CO_PESSOA_FISICA']
b = p15r[['PROF_CO_PESSOA_FISICA','PROF_TP_COR_RACA','PROF_IN_CIDADE_NATAL',\
          'PROF_NUM_TTL_GRAD','PROF_NUM_QTD_ESCOLAS','PROF_NUM_QTD_TURMAS',\
         'PROF_IN_LECIONA_PUBLICA','PROF_IN_LECIONA_PRIVADA']]
c = a.merge(b,on='PROF_CO_PESSOA_FISICA')
#Acabei de descobrir que a cor raca na tabela de matriculas eh a cor/raca do professor 
#e nao dos alunos...

c['TURMA_NUM_ALUNOS_COR_RACA_PROF'] = c.apply(lambda x: None if np.isnan(x['PROF_TP_COR_RACA']) else x['ALUNO_SUM_IN_TP_COR_RACA_'+str(int(x['PROF_TP_COR_RACA']))] ,axis=1)
c['TURMA_PROP_ALUNOS_COR_RACA_PROF'] = c.TURMA_NUM_ALUNOS_COR_RACA_PROF.clip_upper(50) / c.ALUNO_COUNT_CO_PESSOA_FISICA
display(c.TURMA_PROP_ALUNOS_COR_RACA_PROF.value_counts())
c.TURMA_PROP_ALUNOS_COR_RACA_PROF.hist(bins=50);

In [ ]:
pet.columns

In [ ]:
a = turma_alunos.reset_index().merge(profs,on=cols_gb)
a = a[[c for c in a.columns if 'COR_RACA' not in c]]
a.columns = [c for c in a.columns[:-1]] + ['PROF_CO_PESSOA_FISICA']
b = p15r[['PROF_CO_PESSOA_FISICA','PROF_TP_COR_RACA','PROF_IN_CIDADE_NATAL',\
          'PROF_NUM_TTL_GRAD','PROF_NUM_QTD_ESCOLAS','PROF_NUM_QTD_TURMAS',\
         'PROF_IN_LECIONA_PUBLICA','PROF_IN_LECIONA_PRIVADA']]
c = a.merge(b,on='PROF_CO_PESSOA_FISICA')
d = c.groupby(['PROF_CO_PESSOA_FISICA','CO_ENTIDADE'])
del a,b
e = d[['ALUNO_COUNT_CO_PESSOA_FISICA']].mean()
cols_min = ['ALUNO_MEAN_IN_ONIBUS','ALUNO_MEAN_IN_TERRA_NATAL','ALUNO_MEAN_IN_H0MEM','ALUNO_MEAN_NU_IDADE']
f = d[cols_min].min()
f.columns = [f'MIN_{c}' for c in f.columns]
cols_max = cols_min
g = d[cols_max].max()
g.columns = [f'MAX_{c}' for c in g.columns]

model_vars = model_vars.merge(e.reset_index(),left_on=['CO_PESSOA_FISICA', 'CO_ENTIDADE'],right_on=['PROF_CO_PESSOA_FISICA', 'CO_ENTIDADE'])\
.merge(f.reset_index(),left_on=['CO_PESSOA_FISICA', 'CO_ENTIDADE'],right_on=['PROF_CO_PESSOA_FISICA', 'CO_ENTIDADE'])\
.merge(g.reset_index(),left_on=['CO_PESSOA_FISICA', 'CO_ENTIDADE'],right_on=['PROF_CO_PESSOA_FISICA', 'CO_ENTIDADE'])
   

In [ ]:
model_vars = model_vars[[c for c in model_vars.columns if not c.endswith('_x') and not c.endswith('_y')]]

In [ ]:
list(map(print,model_vars.columns));

In [ ]:
model_vars.to_csv(f'{PATH_DF_CSV}model_vars.csv')


<table columns=2 style='width:100%'><tr><td><img src='img/cp_icon.png'/>
</td><td> <h2>Check-point, criação de features da vizinhança da escola a partir dos dados do censo de 2010.</h2></td></tr></table>


In [ ]:
model_vars = pd.read_csv(f'{PATH_DF_CSV}model_vars.csv',index_col=['CO_PESSOA_FISICA','CO_ENTIDADE'])

In [ ]:
escolas = pd.read_csv(f'{CSV_PATH}lclz_df/DADOS_ESCOLAS_PUBLICAS.csv')

In [ ]:
escolas = escolas[escolas.CO_ENTIDADE.isin(model_vars.reset_index().CO_ENTIDADE.unique())]
escolas.shape

In [ ]:
escolas.PRECISAO.value_counts().plot.bar()

In [ ]:
esc = escolas.loc[escolas.PREC_BOA == 1]['CO_ENTIDADE'].values
m = model_vars.reset_index()


In [ ]:
m.loc[m.CO_ENTIDADE.isin(esc)].target.value_counts()

In [ ]:
#parece razoavel essa localizacao, iremos utiliza-la

In [ ]:
model_vars = m.loc[m.CO_ENTIDADE.isin(esc)].set_index(['CO_PESSOA_FISICA','CO_ENTIDADE'])

### Dados das redondezas da escola

In [ ]:
dsc = dados_setor_censitario(f'{CSV_PATH}lclz_df/censo_df/dados/',glob(f'{CSV_PATH}lclz_df/censo_df/*.shp')[0],escolas)

In [ ]:
dsc.head(2).T

In [ ]:
dsc.shape

In [ ]:
dsc.set_index(['CO_ENTIDADE','CO_SETOR_CENSITARIO'],inplace=True)


In [ ]:
for c in dsc.columns:
    display(f'{c} - {int(sum(dsc[c].isna())/dsc.shape[0] * 100)}')
    dsc[c].hist(bins=50)
    plt.show()

As variaveis area media domicilio, densidade demografica e sit setor apresentaram baixa variabilidade, portanto serao retiradas do modelo.

np.nan representam somente 1% da base, portanto tambem serao retirados.

In [ ]:
esc_setores = dsc.copy()

In [ ]:
esc_setores.drop(['area_media_domicilio','densidade_demografica','sit_setor'],axis=1,inplace=True)

In [ ]:
esc_setores.dropna(inplace=True)

In [ ]:
esc_setores.shape

### Agregando informacoes dos setores por escola

In [ ]:
esc_setores.reset_index(inplace=True)
esc_setores.columns

In [ ]:
e = esc_setores.groupby('CO_ENTIDADE')


In [ ]:
qtds = e[['CO_SETOR_CENSITARIO']].nunique()
qtds.columns = [f'QTD_{col}' for col in qtds.columns ]
somas = e[[col for col in esc_setores.columns if col.startswith('qtd')]].sum()
somas.columns = [f'SUM_{col}' for col in somas.columns ]
maximos = e[[col for col in esc_setores.columns if col.startswith('qtd')] + ['renda_media_dom','DISTANCIA_SETOR_ENTIDADE']].max()
maximos.columns = [f'MAX_{col}' for col in maximos.columns ]
minimos = e[[col for col in esc_setores.columns if col.startswith('qtd')] + ['renda_media_dom','DISTANCIA_SETOR_ENTIDADE']].min()
minimos.columns = [f'MIN_{col}' for col in minimos.columns ]


In [ ]:
md = model_vars.reset_index().merge(qtds,left_on='CO_ENTIDADE',right_index=True)\
.merge(somas,left_on='CO_ENTIDADE',right_index=True)\
.merge(maximos,left_on='CO_ENTIDADE',right_index=True)\
.merge(minimos,left_on='CO_ENTIDADE',right_index=True)

In [ ]:
model_vars = md.set_index(['CO_PESSOA_FISICA', 'CO_ENTIDADE'])

In [ ]:
[print(col) for col in model_vars.columns];

In [ ]:
display(model_vars.groupby('target')[model_vars.columns[0]].count())
display(model_vars.shape)

In [ ]:
model_vars = model_vars[[c for c in model_vars.columns[1:]]]
model_vars.head()

In [ ]:
escolas = ajusta_colunas_int_df_inep(pd.read_csv(f'{PATH_DF_CSV}escolas_distritais_15_17_func_df.csv',low_memory=False),vai_printar_cols=False)
model_vars = model_vars.merge(escolas.loc[escolas.NU_ANO_CENSO==2015][['NO_ENTIDADE','CO_ENTIDADE']],on='CO_ENTIDADE')


In [ ]:
ras = pd.read_csv(f'{CSV_PATH}lclz_df/RA_DF.csv',index_col='cod')
model_vars = model_vars.merge(ras,left_on='CO_ORGAO_REGIONAL',right_index=True)


In [ ]:
model_vars.to_csv(f'{PATH_DF_CSV}model_vars.csv')

Ainda poderiamos acrescentar dados de: 
- Torre de celular 
- Minha casa, minha vida
- Areas em risco de alagamento
- Criminalidade por Regiao Administrativa
Mas vamos continuar, caso o modelo nao performe acrescentaremos essas features.

### <center> Fim da geração de features, início da investigação </center>

<img src='https://media.giphy.com/media/3orieUe6ejxSFxYCXe/giphy.gif'/>

<table><tr><td><img src='img/cp_icon.png'/></td><td>

## Checkpoint</td></tr>



In [ ]:
model_vars = pd.read_csv(f'{PATH_DF_CSV}model_vars.csv',low_memory=False)
display(model_vars.head())

<img src='img/exploracao.gif'/>

# Exploração dos dados obtidos

### Quais sao as escolas com maior evasao de professores e aonde elas ficam?

In [ ]:
a = model_vars.groupby('CO_ENTIDADE')[['target']].sum()
b = model_vars.groupby('CO_ENTIDADE')[['CO_PESSOA_FISICA']].count()
c = model_vars[['CO_ENTIDADE','LAT','LONG','dcr','NO_ENTIDADE','CO_ORGAO_REGIONAL']].drop_duplicates()
d = a.merge(b,on='CO_ENTIDADE').merge(c,on='CO_ENTIDADE')
d['prop_saida'] = d.target/d.CO_PESSOA_FISICA * 100
d.sort_values('prop_saida',ascending=False,inplace=True)
d.head(10)

In [ ]:
display(d.prop_saida.describe())
d.prop_saida.hist();

Parece que a partir do percentil 75 a _coisa é feia_ pois mais da metade dos professores saíram. 

Onde ficam essas escolas?

In [ ]:
display(d.CO_PESSOA_FISICA.describe([.1,.25,.50,.75,.9]));
d.CO_PESSOA_FISICA.hist();

In [ ]:
import folium

In [ ]:
lclz_praca_3_poderes = [-15.7994, -47.8613]
map = folium.Map(location=lclz_praca_3_poderes, zoom_start=10)
locations = d.loc[d.CO_PESSOA_FISICA >= 7][['LAT', 'LONG']].head(20)
locationlist = locations.values.tolist()

In [ ]:
display(d.loc[d.CO_PESSOA_FISICA >= 7].head(20))
locationlist


Pesquisei essas localizações no Google Maps e as algumas localizações não foram exatas mas deram uma aproximação aceitável (o pior caso ficou 6 quarteirões de distância).

In [ ]:
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point]).add_to(map)
map

Pontos de interessantes:
- Como explicar as saídas de escolas do Plano Piloto (Brasília)? Seriam elas exceções? 

Vamos plotar um mapa de calor das 50 piores e ver o que acontece.

In [ ]:
#vamos tirar o 100% porque deve ter acontecido alguma coisa com essa escola
data = d.loc[(d.CO_PESSOA_FISICA >= 7) & (d.prop_saida < 100)][['LAT', 'LONG', 'prop_saida']].head(50).values.tolist()


In [ ]:
from folium.plugins import HeatMap
m = folium.Map(lclz_praca_3_poderes)
HeatMap(data).add_to(m)
m

Mais ou menos como esperado:
    - algumas áreas populosas e mais afastadas do centro estão bem destacadas.
    - Não esperava que o centro de Brasília fosse ter tanto destaque.
    
Assim temos algumas pistas que renda nas imediações nao tem muito a ver (Plano Pilo e Lago Sul). Vamos comprovar  
    

In [ ]:
model_vars.MAX_qtd_dom_renda_max_per_mais_5sm.hist(bins=20)

In [ ]:
model_vars.loc[model_vars.target == 0].MAX_qtd_dom_renda_max_per_mais_5sm.hist(bins=20);

In [ ]:
model_vars.loc[model_vars.target == 1].MAX_qtd_dom_renda_max_per_mais_5sm.hist(bins=20);

In [ ]:
model_vars[['MAX_qtd_dom_renda_max_per_mais_5sm','target']].corr()

A correlação diz aquilo que pode-se ver nos gráficos: a renda menor influencia muito pouco na mudança de escola.

#### Vamos ver todas as correlações com a target

In [ ]:
cols_alta_correlacao =[]
for l in model_vars.drop('PROF_CO_PESSOA_FISICA',axis=1).set_index(['CO_PESSOA_FISICA','CO_ENTIDADE']).corr()[['target']].sort_values('target',ascending=False).reset_index().itertuples():
    display(str(l[1]) + ' ' + str(l[2]))
    if l[2] < -0.02 or l[2] > 0.02:
        cols_alta_correlacao.append(l[1])

### Uma modelagem com Random Forest para verificar as features mais importantes.

In [ ]:
a = processa_tudo(model_vars[cols_alta_correlacao])

In [ ]:
b = processa_tudo(model_vars[a[0][:10]+ ['target']])

In [ ]:
b = processa_tudo(model_vars[b[0][:3]+ ['target']])

<img src='https://media.giphy.com/media/3o6Mb2JwJPVu7VMDSw/giphy.gif'/>

#### Esse resultado está muito ruim (claramente com _overfitting_), vamos fazer uns ajustes com cross validation e grid search.

In [ ]:
cols_drop = ['PROF_CO_PESSOA_FISICA','NO_ENTIDADE','dcr','LAT','LONG']
a = processa_tudo(model_vars.drop(cols_drop,axis=1).set_index(['CO_PESSOA_FISICA','CO_ENTIDADE']),True),

In [ ]:
fi = a[0][0]
clf = a[0][1]

In [ ]:
md_b = model_vars.set_index(['CO_PESSOA_FISICA','CO_ENTIDADE'])[fi[:50] + ['target']]
b = processa_tudo(md_b,True),

In [ ]:
clf = b[0][1]

### Consideracoes:
01. Com Grid Search a performance melhora consideravelmente.

02. O resultado retirando uma amostra da base para balancear (com o ```sample``` do pandas ) ficou estranho, resultado muito bom no teste e ruim no treino e quando aplica-se a todo o dataset.

### Com esse resultado estamos assim:
<!--
<img src='img/homer-simpson-woohoo-gif-6.gif'/>
-->

## Primeiro, vamos analisar quando o classificador erra.

In [ ]:
y_true = md_b.target
X = md_b.drop('target',axis=1).values
y_pred = clf.predict(X)
lc = [c for y1,y2,c in zip(y_true,y_pred,md_b.index) if y1 != y2]

In [ ]:
1 - len(lc)/len(md_b.index)